# Illustrating a commitment scheme based on symmetric encryption
Implementation of the intuitive approach described by Schneier. 

Cf. Schneier, B. 2007. Applied cryptography: protocols, algorithms, and source code in C. john wiley & sons, S.86f.

In [1]:
import numpy as np
from numpy.random import RandomState
from Crypto.Cipher import AES
from Crypto import Random

In [34]:
class Alice(object):
    """
    Implements the role of Alice, who commits to a bit with the means of symmetric encryption 
    and reveals her choice some time later. 
    """
    def __init__(self):
        self._cipher_params = None
    
    def commit(self, nonce):
        chosen_value = np.random.randint(0xFFFF)
        print('Alice choice: ', chosen_value)
        n = 24  # key length
        n_iv = 16  # length of IV
        key = Random.get_random_bytes(n)
        iv = Random.get_random_bytes(n_iv)
        obj = AES.new(key, AES.MODE_CBC, iv)

        message = str(chosen_value) + str(nonce)
        len_padding = 0

        while len(message) % 16 != 0:
            message += '.'
            len_padding += 1

        cipher_text = obj.encrypt(message)
        self._cipher_params = [iv, key, len_padding]

        return cipher_text
    
    def reveal(self):
        return self._cipher_params

In [40]:
"""This simulates Bob"""
def decrypt(cipher_text, iv, key, len_padding):
    obj = AES.new(key, AES.MODE_CBC, iv)
    recovered_plaintext = obj.decrypt(cipher_text)[:-len_padding]
    return recovered_plaintext
    
# Hello Alice!
alice = Alice()

# Bob chooses a random number
rand = np.random.randint(0xFFFF)
print('Bob\'s choice: ', rand)

# Alice, commit to a bit!
commitment = alice.commit(rand)

# Alice, please reveal your choice
cipher_params = alice.reveal()
plaintext = decrypt(commitment, *cipher_params)

# Bob checks validity of commitment by evaluating the existence of his random number  
if rand == int(plaintext[-len(str(rand)):]):
    print('----------------------------------')
    print('Commitment is valid')
    print('Alice choice was: ', int(decrypted_commitment[:len(rand_str)]))

Bob's choice:  21438
Alice choice:  29250
----------------------------------
Commitment is valid
Alice choice was:  3572
